In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
train_data_dir = 'data/'

In [10]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 10
num_classes = 5

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 2931 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [5]:
def build_model(input_size,nb_classes):
  inputs = Input(input_size)

  conv1 = Conv2D(8, 3, input_shape=input_size, activation='relu', padding='same',
                         kernel_initializer='he_normal')(inputs)
  conv2 = Conv2D(8, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
  merge1 = concatenate([conv1, conv2], axis=3)

  mp1 = MaxPooling2D((2,2))(merge1)
  drop1 = Dropout(0.2)(mp1)

  conv3 = Conv2D(8, 3, activation='relu', padding='same',
                        kernel_initializer='he_normal')(drop1)
  conv4 = Conv2D(8, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
  merge2 = concatenate([conv3, conv4], axis=3)

  mp2 = MaxPooling2D((2, 2))(merge2)
  drop2 = Dropout(0.35)(mp2)
  conv5 = Conv2D(16, 3, activation='relu', padding='same',
                        kernel_initializer='he_normal')(drop2)
  conv6 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
  merge3 = concatenate([conv5, conv6], axis=3)

  mp3 = MaxPooling2D((2, 2))(merge3)

  conv7 = Conv2D(16, 3, activation='relu', padding='same',
                        kernel_initializer='he_normal')(mp3)
  conv8 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
  merge3 = concatenate([conv7, conv8], axis=3)

  conv9 = Conv2D(32, 3, activation='relu', padding='same',
                  kernel_initializer='he_normal')(merge3)
  conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
  merge4 = concatenate([conv9, conv10], axis=3)

  mp4 = MaxPooling2D((2, 2))(merge4)

  conv11 = Conv2D(32, 3, activation='relu', padding='same',
                  kernel_initializer='he_normal')(mp4)
  conv12 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv11)
  merge5 = concatenate([conv11, conv12], axis=3)

  conv13 = Conv2D(64, 3, activation='relu', padding='same',
                  kernel_initializer='he_normal')(merge5)
  conv14 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv13)
  merge6 = concatenate([conv13, conv14], axis=3)

  conv15 = Conv2D(64, 3, activation='relu', padding='same',
                  kernel_initializer='he_normal')(merge6)
  conv16 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv15)
  merge7 = concatenate([conv15, conv16], axis=3)

  mp5 = MaxPooling2D((2, 2))(merge7)

  drop4 = Dropout(0.45)(mp5)
  flat = Flatten()(drop4)

  dense1 = Dense(nb_classes, activation='softmax')(flat)

  return Model(inputs, dense1)

In [6]:
model = build_model((img_width, img_height , 3) , num_classes)

In [7]:
checkpoint_filepath = 'model.h5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)
stopper = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
)

In [8]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 8)  224         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 8)  584         ['conv2d[0][0]']                 
                                                                                                  
 concatenate (Concatenate)      (None, 224, 224, 16  0           ['conv2d[0][0]',             

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint , stopper]
)

Epoch 1/10
51/91 [===============>..............] - ETA: 26s - loss: 7.9266 - accuracy: 0.4534   

In [ ]:
model.save('model.h5')

In [ ]:
from pickle import dump
with open('classes.h5' , 'wb') as f:
    dump(train_generator.class_indices , f)